# Урок 3. Логистическая регрессия. Log Loss

In [74]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%matplotlib inline
plt.style.use('seaborn-ticks')
plt.rcParams.update({'font.size': 14})

In [57]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],  # квалификация репетитора
              dtype = np.float64).T 

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1]) # поступил или нет ученик на специальность Математика
X

array([[1.00e+00, 1.00e+00, 5.00e+02, 1.00e+00],
       [1.00e+00, 1.00e+00, 7.00e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 7.50e+02, 2.00e+00],
       [1.00e+00, 1.00e+00, 6.00e+02, 1.00e+00],
       [1.00e+00, 3.00e+00, 1.45e+03, 2.00e+00],
       [1.00e+00, 0.00e+00, 8.00e+02, 1.00e+00],
       [1.00e+00, 5.00e+00, 1.50e+03, 3.00e+00],
       [1.00e+00, 1.00e+01, 2.00e+03, 3.00e+00],
       [1.00e+00, 1.00e+00, 4.50e+02, 1.00e+00],
       [1.00e+00, 2.00e+00, 1.00e+03, 2.00e+00]])

In [58]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [59]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [60]:
X_st

array([[ 1.00000000e+00,  1.00000000e+00,  5.00000000e+02,
         1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  7.00000000e+02,
         1.00000000e+00],
       [-5.79407182e-01, -5.76321125e-01,  1.73204943e+00,
        -5.76321125e-01],
       [ 1.00000000e+00,  1.00000000e+00,  6.00000000e+02,
         1.00000000e+00],
       [ 1.00000000e+00,  3.00000000e+00,  1.45000000e+03,
         2.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  8.00000000e+02,
         1.00000000e+00],
       [ 1.00000000e+00,  5.00000000e+00,  1.50000000e+03,
         3.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+01,  2.00000000e+03,
         3.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  4.50000000e+02,
         1.00000000e+00],
       [ 1.00000000e+00,  2.00000000e+00,  1.00000000e+03,
         2.00000000e+00]])

In [61]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

In [62]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

__Logistic Regression__

In [63]:
def standard_scaler(x):
    res = (x - x.mean()) / x.std()
    return res

X_st = X.copy()
X_st[:, 2] = standard_scaler(X[:, 2])
X_st[:, 2]

array([-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
       -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735])

In [87]:
def eval_LR_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * (X.T @ (y_pred - y)))
        if i % (iterations / 10) == 0:
            print(f'Итерация = {i}, Веса = {w}, LogLoss = {err}')
    return w

In [88]:
w = eval_LR_model(X_st, y, 2000, 1e-2)

Итерация = 200, Веса = [ 0.14805533 -0.69317972  0.77965488  1.21287192], LogLoss = 0.5687278067099054
Итерация = 400, Веса = [-0.00887935 -0.68415387  0.90713736  1.22443854], LogLoss = 0.5476800278385013
Итерация = 600, Веса = [-0.1239972  -0.69202069  0.98318257  1.26924759], LogLoss = 0.5369841864977791
Итерация = 800, Веса = [-0.21681092 -0.7078635   1.03062761  1.32973537], LogLoss = 0.5295551503580799
Итерация = 1000, Веса = [-0.29764618 -0.72670545  1.061634    1.39666497], LogLoss = 0.5233765331724747
Итерация = 1200, Веса = [-0.37188064 -0.7461597   1.08260067  1.46531853], LogLoss = 0.5178498846713239
Итерация = 1400, Веса = [-0.44230332 -0.76510554  1.09696619  1.53338645], LogLoss = 0.5127677838840146
Итерация = 1600, Веса = [-0.51036921 -0.783047    1.10664485  1.59979796], LogLoss = 0.5080360394424425
Итерация = 1800, Веса = [-0.57684595 -0.79979671  1.11275552  1.66410902], LogLoss = 0.503597783659264
Итерация = 2000, Веса = [-0.64214517 -0.81531815  1.11599369  1.72618

In [12]:
y_pred = sigmoid(np.dot(X_st, w))
print(y_pred.round(2))
print(y)
calc_logloss(y, y_pred)

[0.3  0.41 0.66 0.36 0.81 0.66 0.84 0.22 0.28 0.78]
[0 0 1 0 1 0 1 0 1 1]


0.4993913630405437

## Домашнее задание

__1. Измените функцию `calc_logloss` так, чтобы нули по возможности не попадали в `np.log` (как вариант - использовать `np.clip` или `np.where`).__

In [13]:
# добавляем условия
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred, where=(y_pred!=0)) + (1.0 - y) * np.log(1.0 - y_pred, where=(1-y_pred!=0)))
    err = np.sum(err)
    return err

  
__2. Подберите аргументы функции `eval_LR_model` для логистической регрессии таким образом, чтобы log loss был минимальным. Покажите влияние гиперпараметров на ошибку алгоритма (оптимально здесь использовать критерий остановки обучения).__

In [29]:
%%time
min_err = np.inf
min_alpha = ()
min_iters = ()

for alpha in [1e-1, 1e-2, 1e-3, 1e-4]:
    for iters in [1e2, 1e3, 1e4, 1e5]:
        k, num, wi, err = eval_LR_model(X_st, y, int(iters), alpha)
        print(f'Альфа = {alpha}')
#         if err < min_err and (iters % 100 == 0):
#             min_err = err
#             min_alpha = alpha
#             min_iters = iters
     
#         if err[wi] > err[wi-1]:
#             next(iters)
#         else:
#             break
#             print(f'Скорость обучения (альфа) = {alpha} | количество итераций = {iters}\n'
#             f'logLoss: {err} | Подобранные веса = {w}\n')

Итерация = 10, Веса = [ 0.25404989 -0.69257176  0.68914781  1.23515494], LogLoss = 0.5925316168396398
Итерация = 20, Веса = [ 0.14594713 -0.69347808  0.78221643  1.2114552 ], LogLoss = 0.5697629147350483
Итерация = 30, Веса = [ 0.06051315 -0.68552581  0.85465728  1.21171762], LogLoss = 0.5566325734764129
Итерация = 40, Веса = [-0.01086542 -0.68365659  0.90991129  1.22344049], LogLoss = 0.5480608521548668
Итерация = 50, Веса = [-0.07197272 -0.68623725  0.95242318  1.24324378], LogLoss = 0.5419672426338861
Итерация = 60, Веса = [-0.12559098 -0.69176936  0.98552719  1.26867802], LogLoss = 0.5372341719773465
Итерация = 70, Веса = [-0.17376597 -0.69921254  1.01164064  1.29791429], LogLoss = 0.5332724988195368
Итерация = 80, Веса = [-0.21797982 -0.70784853  1.03250384  1.32962077], LogLoss = 0.5297726356017475
Итерация = 90, Веса = [-0.25929815 -0.71718124  1.0493691   1.36284358], LogLoss = 0.5265703859163215
Итерация = 100, Веса = [-0.29848458 -0.72686866  1.06314016  1.39690737], LogLoss 

Итерация = 90000, Веса = [-10.67189098  -1.40567714  -2.19489264   9.01414008], LogLoss = 0.25866230495923226
Итерация = 100000, Веса = [-11.27192801  -1.45338517  -2.38300258   9.49385018], LogLoss = 0.2523828537754709
Альфа = 0.01
Итерация = 10, Веса = [ 0.49282652 -0.1500769   0.64749017  1.51727788], LogLoss = 1.2031627390407214
Итерация = 20, Веса = [ 0.48896027 -0.1618525   0.64728215  1.51155479], LogLoss = 1.1845113053551715
Итерация = 30, Веса = [ 0.48511584 -0.17358898  0.64706476  1.50586157], LogLoss = 1.1660019273183884
Итерация = 40, Веса = [ 0.4812937  -0.185284    0.64683835  1.50019927], LogLoss = 1.147640985999691
Итерация = 50, Веса = [ 0.47749435 -0.196935    0.6466033   1.49456902], LogLoss = 1.1294354248962641
Итерация = 60, Веса = [ 0.47371831 -0.20853915  0.64636008  1.488972  ], LogLoss = 1.1113928003861757
Итерация = 70, Веса = [ 0.46996611 -0.22009334  0.64610919  1.48340953], LogLoss = 1.0935213331749436
Итерация = 80, Веса = [ 0.46623834 -0.23159416  0.6458

Итерация = 70000, Веса = [-0.17224326 -0.69935523  1.00931153  1.29828743], LogLoss = 0.5330172710285223
Итерация = 80000, Веса = [-0.21668343 -0.70786696  1.03042468  1.32975123], LogLoss = 0.5295310331283415
Итерация = 90000, Веса = [-0.25820024 -0.71709221  1.04751085  1.36276018], LogLoss = 0.5263390430000646
Итерация = 100000, Веса = [-0.29755454 -0.72668938  1.06147096  1.3966418 ], LogLoss = 0.5233543557317326
Альфа = 0.0001
Wall time: 17.8 s


Минимальниый LogLoss получен при следующих параметрах:

In [30]:
W = eval_LR_model(X_st, y, 500000, 1e-1)

Итерация = 50000, Веса = [-26.07030122  -2.7286846   -6.81038328  21.47869092], LogLoss = 0.15099538645603025
Итерация = 100000, Веса = [-35.8933164   -3.59426199  -9.64639296  29.4060647 ], LogLoss = 0.11667060791100063
Итерация = 150000, Веса = [-42.58016954  -4.1712215  -11.56630967  34.75695793], LogLoss = 0.10108464416583551
Итерация = 200000, Веса = [-47.83523093  -4.60958703 -13.08107424  38.92379106], LogLoss = 0.09156004715343083
Итерация = 250000, Веса = [-52.27867366  -4.96595203 -14.37104267  42.41473228], LogLoss = 0.08480374722276789
Итерация = 300000, Веса = [-56.20163494  -5.26790425 -15.51900064  45.4697445 ], LogLoss = 0.07957197301851478
Итерация = 350000, Веса = [-59.76216037  -5.53122724 -16.5688773   48.22042244], LogLoss = 0.07528611324547611
Итерация = 400000, Веса = [-63.05440358  -5.76596608 -17.54620491  50.74625575], LogLoss = 0.07163868553431933
Итерация = 450000, Веса = [-66.13822395  -5.97896062 -18.46682121  53.09854893], LogLoss = 0.06845043440796936
Ит

__3. Создайте функцию `calc_pred_proba`, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred_proba`).__

In [68]:
def calc_pred_proba(X, W):
    y_pred_proba = np.floor(sigmoid(X @ W))
    return y_pred_proba

In [69]:
calc_pred_proba(X, W)

C:\Users\demin\AppData\Local\Temp/ipykernel_10124/2870899410.py:2: RuntimeWarning: overflow encountered in exp
  res = 1 / (1 + np.exp(-z))


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

__4. Создайте функцию `calc_pred`, возвращающую предсказанный класс (на вход подаются значения признаков `Х` и веса, которые уже посчитаны функцией `eval_LR_model`, на выходе - массив `y_pred`).__

In [66]:
def calc_pred(W, X, threshold):
    z = X @ W
    y_pred = sigmoid(z)
    y_pred[y_pred > threshold] = 1
    y_pred[y_pred < 1] = 0
    return  y_pred

In [67]:
y_pred = calc_pred(W, X_st, 0.5)
y_pred

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

__5. Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.__

In [70]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

In [71]:
# Accuracy
def accuracy_(y_pred, y):
    return len(y_pred[y == y_pred]) / len(y_pred)

accuracy_(y_pred, y)

1.0

In [72]:
accuracy_score(y, y_pred)

1.0

In [75]:
# Correlation matrix
def correlation_matrix(y_pred, y):
    TP = len(y_pred[(y == y_pred) & (y_pred == 1)])
    FN = len(y_pred[(y != y_pred) & (y_pred == 0)])
    FP = len(y_pred[(y != y_pred) & (y_pred == 1)])
    TN = len(y_pred[(y == y_pred) & (y_pred == 0)])
    corr_matrix = pd.DataFrame.from_dict(
        {'y_pred': ['a(x) = +1', 'a(x) = -1'], 
         'y = +1': [TP, FN], 
         'y = -1': [FP, TN]}).set_index('y_pred')
    return corr_matrix

correlation_matrix(y_pred, y)

,y = +1,y = -1
y_pred,,
a(x) = +1,5,0
a(x) = -1,0,5


In [76]:
confusion_matrix(y, y_pred)

array([[5, 0],
       [0, 5]], dtype=int64)

In [77]:
# Precision
def precision_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 1)]))

precision_(y_pred, y)

1.0

In [78]:
precision_score(y, y_pred)

1.0

In [79]:
# Recall
def recall_(y_pred, y):
    return len(y_pred[(y == y_pred) & (y_pred == 1)]) / (
        len(y_pred[(y == y_pred) & (y_pred == 1)]) + len(y_pred[(y != y_pred) & (y_pred == 0)]))

recall_(y_pred, y)

1.0

In [80]:
recall_score(y, y_pred)

1.0

In [81]:
# F1-score
def F1_(y_pred, y):
    return 2 * precision_(y_pred, y) * recall_(y_pred, y) / (precision_(y_pred, y) + recall_(y_pred, y))

F1_(y_pred, y)

1.0

In [82]:
f1_score(y, y_pred)

1.0

__6. Могла ли модель переобучиться? Почему?__

Да могла. Модель переобучилась из-за малого количества данных.

__7. (\*) Создайте функции `eval_LR_model_l1` и `eval_LR_model_l2` с применением L1 и L2 регуляризации соответственно.__

In [92]:
# L1
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-6):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(W @ X)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * ((y_pred - y) @ X.T)  + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

eval_model_l1(X, y, iterations=1000, alpha=1e-4, lambda_=1e-6)

ValueError: operands could not be broadcast together with shapes (10,) (4,) 

In [86]:
# L2
def eval_model_l2(X, y, iterations, alpha=1e-6, lambda_=1e-7):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        y_pred = sigmoid(np.dot(W, X))
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

eval_model_l2(X_st, y, iterations=1000, alpha=1e-5, lambda_=1e-6)

ValueError: operands could not be broadcast together with shapes (10,) (4,) 

Методичка (исправленная) https://colab.research.google.com/drive/1DxLcLdf2Lns12qOqGVvapDha5tiltM1f?usp=sharing